# Task 3.1

### Use the Notebook to build the code to scrape the following Wikipedia page
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown

In [113]:
!pip install BeautifulSoup4

In [114]:
!pip install geopy

### Import all packages and libs

In [115]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import csv
import requests
!pip install folium
import folium

In [116]:
html=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=html[0]

### Dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [118]:
df.columns=["PostalCode","Borough","Neighborhood"]
df=df[df["Borough"]!= "Not assigned"]
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [120]:
post=df["PostalCode"].unique()
df.set_index("PostalCode", drop=False, inplace=True)
sign = ", "


In [121]:
for i in post:
    x=df.loc[i]
    if x.size>3:
        b=sign.join(x["Neighborhood"])
        df.loc[i, "Neighborhood"]=b
df.drop_duplicates('PostalCode', inplace=True)
df.reset_index(drop = True, inplace=True)
df.iloc[0:12]

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood to be the same as the borough. 

In [123]:
df=df[df["Borough"]!= "Not assigned"]
df.set_index("PostalCode", drop=False, inplace=True)
df.loc["M9V"]["Borough"]=df.loc["M9V"]["Neighborhood"]="Queen's Park"
df.reset_index(drop = True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [124]:
print("The shape of this dataframe is "+ str(df.shape))

The shape of this dataframe is (103, 3)


# EXERCISE 3.2

### Using Geocoder package
Link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [126]:
!wget -q -O "geospatialdata.csv" https://cocl.us/Geospatial_data
df_geospatial = pd.read_csv("geospatialdata.csv")
df_geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [127]:
df_geospatial.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df_geo=pd.merge(df,df_geospatial[["PostalCode", "Latitude", "Longitude"]], on="PostalCode")
df_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


# Exercises 3.3

In [128]:
df_toronto=df_geo[df_geo["Borough"].str.contains("Toronto")].reset_index(drop=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
